# 模型导出与部署

## 连接h2o

In [1]:
library(h2o)


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: 'h2o'

The following objects are masked from 'package:stats':

    cor, sd, var

The following objects are masked from 'package:base':

    %*%, %in%, &&, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



In [2]:
h2o.init(startH2O = FALSE)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 21 minutes 
    H2O cluster timezone:       Asia/Shanghai 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.20.0.3 
    H2O cluster version age:    1 month and 24 days  
    H2O cluster name:           h2o-rest 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   0.73 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.5.1 (2018-07-02) 



In [4]:
getwd()

[1] "E:/B-jupyter-wd/h2o-tutorial"

## 导入数据训练模型

In [5]:
df <- h2o.importFile(normalizePath("./data/prostate.csv"))
h2o.describe(df)

  |======================================================================| 100%


Label,Type,Missing,Zeros,PosInf,NegInf,Min,Max,Mean,Sigma,Cardinality
ID,int,0,0,0,0,1.0,380.0,190.5000000,109.8407939,NA
CAPSULE,int,0,227,0,0,0.0,1.0,0.4026316,0.4910743,NA
AGE,int,0,0,0,0,43.0,79.0,66.0394737,6.5270713,NA
RACE,int,0,3,0,0,0.0,2.0,1.0868421,0.3087733,NA
DPROS,int,0,0,0,0,1.0,4.0,2.2710526,1.0001076,NA
DCAPS,int,0,0,0,0,1.0,2.0,1.1078947,0.3106564,NA
PSA,real,0,0,0,0,0.3,139.7,15.4086316,19.9975727,NA
VOL,real,0,167,0,0,0.0,97.6,15.8129211,18.3476200,NA
GLEASON,int,0,2,0,0,0.0,9.0,6.3842105,1.0919534,NA


In [10]:
df$CAPSULE <- as.factor(df$CAPSULE)
model <- h2o.gbm(y="CAPSULE",
        x=c("AGE", "RACE", "PSA", "GLEASON"),
        training_frame=df,
        distribution="bernoulli",
        ntrees=100,
        max_depth=4,
        learn_rate=0.1)

  |======================================================================| 100%


In [12]:
if(!dir.exists("./experiments")) dir.create("./experiments")

In [15]:
summary(model)

Model Details:

H2OBinomialModel: gbm
Model Key:  GBM_model_R_1536024753790_103 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             100                      100               18758         4
  max_depth mean_depth min_leaves max_leaves mean_leaves
1         4    4.00000          5         15     9.99000

H2OBinomialMetrics: gbm
** Reported on training data. **

MSE:  0.09537208
RMSE:  0.3088237
LogLoss:  0.3198346
Mean Per-Class Error:  0.1224123
AUC:  0.9538309
Gini:  0.9076617

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
         0   1    Error     Rate
0      210  17 0.074890  =17/227
1       26 127 0.169935  =26/153
Totals 236 144 0.113158  =43/380

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.459543 0.855219 140
2                       max f2  0.265691 0.903614 211
3                 max f

## 模型结果导出

In [19]:
h2o.download_mojo(model, path = file.path(getwd(),"experiments"),get_genmodel_jar = TRUE)

[1] "Model saved to GBM_model_R_1536024753790_103.zip"


In [21]:
h2o.download_pojo(model,path = file.path(getwd(),"experiments"))

[1] "GBM_model_R_1536024753790_103.java"

## 部署说明

h2o提供两种部署方式：MOJO和POJO。

MOJO: 适用于复杂模型，例如gbm,xgboost, ensamble model 等，需要依赖模型生成的文件。    
POJO: 适用于分类和回归模型，以Java class的形式调用，不能超过1G，较简单的模型比mojo速度快。
